In [1]:
from pymatgen.ext.matproj import MPRester

/home/zeng/yinglu/.conda/envs/my_pymatgen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pymatgen.io.vasp.inputs as inp

In [3]:
import pymatgen.io.vasp.sets as sets

In [4]:
from pymatgen.io.vasp.sets import MPRelaxSet

In [5]:
import numpy as np

In [6]:
import os

In [7]:
import subprocess

In [8]:
# here the time should given user the permission to send parameters.
def sbatch(
    job_name, command, work_dir, time=48, mem=1, nodes=1, tasks=1, partition="batch", dep=""
):
    # jobs depend on the results of former jobs
    print("dep is:")
    print(dep)
    if dep != "":
        dep = "--dependency=afterok:{} --kill-on-invalid-dep=yes ".format(dep)
        # print(dep)

    # prepare a submitting file
    print("The current directory is:")
    print(os.getcwd())
    print("The work directory is:")
    print(work_dir)
    file = open(work_dir + "/sub.sh", "w")
    headline = "#!/bin/bash\n#SBATCH --time={}:00:00\n#SBATCH --mem-per-cpu={}G\n#SBATCH --job-name={}\n#SBATCH --error={}.err\n#SBATCH --output={}.out\n#SBATCH --nodes={}\n#SBATCH --ntasks-per-node={}\n#SBATCH --partition={}\n#SBATCH {}\n".format(
        time, mem, job_name, job_name, job_name, nodes, tasks, partition, dep
    )
    loadline = "module load compiler/intel/19 openmpi/4.1 vasp/6.2\n"

    comline = command

    # Writing a string to file
    file.write(headline)
    file.write(loadline)
    file.write(comline)

    # Closing file
    file.close()

    sbatch_command = "sbatch --parsable sub.sh"
    sbatch_response = subprocess.getoutput(sbatch_command)
    job_id = sbatch_response.split(" ")[-1].strip()

    return job_id

In [9]:
work_dir = "YOUR OWN WORKDIR"
os.chdir(work_dir)

In [10]:
with MPRester("YOUR OWN PASSWORD") as m:
    chemicals = ["Pt", "PtCl4", "Sn", "SnCl2", "Bi", "BiCl3", "Mn", "MnCl2", "Cu", "CuCl2", "La", "LaCl3", "Pd", "PdCl2", "Mg", "MgCl2", "Ta", "TaCl5", "Hf", "HfCl4"]

    for chemical in chemicals:
        # find all of the ones that has a composition given. The data needed are only "composition", "material_id", "energy_above_hull"
        print(chemical)
        docs = m.summary.search(formula=chemical, fields = ["composition", "material_id", "energy_above_hull", 'theoretical'])
        print(len(docs))

        # decide the structure that contains the smallest energy based on the convex hull energy
        energy = np.zeros(len(docs))
        for i in np.arange(len(docs)):
            energy[i] = docs[i].energy_above_hull
        # get the structure
        min_index = energy.argmin()
        print(docs[min_index].energy_above_hull)
        structure = m.get_structure_by_material_id(docs[min_index].material_id)
        
        # then create a vasp job to run the calculation
        # make a directory
        chemical_dir = work_dir + "/" + chemical
        if not os.path.exists(chemical_dir):
            os.makedirs(chemical_dir)

        my_custom_input_set = MPRelaxSet(structure, user_incar_settings={"EDIFF":1E-6, "EDIFFG":-0.02, "ISMEAR":0, "NSW": 200, "NPAR":6}, user_kpoints_settings={"reciprocal_density":100}, user_potcar_functional='PBE_52')
        my_custom_input_set.write_input(chemical_dir)
        os.chdir(chemical_dir)
        command = "mpirun vasp_std"
        job_ID = sbatch(job_name=chemical, command = command, work_dir = chemical_dir, tasks=36)
        print(job_ID)
        os.chdir(work_dir)

/home/zeng/yinglu/.conda/envs/my_pymatgen/lib/python3.11/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Ca


Retrieving SummaryDoc documents: 100%|██████████| 12/12 [00:00<00:00, 320583.75it/s]


12
0.0


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 26214.40it/s]
/home/zeng/yinglu/.conda/envs/my_pymatgen/lib/python3.11/site-packages/pymatgen/io/vasp/sets.py:408: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(
/home/zeng/yinglu/.conda/envs/my_pymatgen/lib/python3.11/site-packages/pymatgen/io/vasp/sets.py:591: BadInputSetWarning: Relaxation of likely metal with ISMEAR < 1 detected. Please see VASP recommendations on ISMEAR for metals.
  warnings.warn(


dep is:

The current directory is:
/lustre/work/zeng/yinglu/project/cai2/energy/Ca
The work directory is:
/work/zeng/yinglu/project/cai2/energy/Ca
3036267
CaCl2


Retrieving SummaryDoc documents: 100%|██████████| 5/5 [00:00<00:00, 165130.08it/s]


5
0.0


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 19418.07it/s]

dep is:

The current directory is:
/lustre/work/zeng/yinglu/project/cai2/energy/CaCl2
The work directory is:
/work/zeng/yinglu/project/cai2/energy/CaCl2
3036268
